In [13]:
class KatoStrategy(object):
    def __init__(self, random_state=None):
        if random_state is None:
            random_state = np.random.RandomState()
        self.random_state = random_state

        self.my_history = []
        
        self.signals = []
        
    def normal_action(self):
        if len(self.signals)%3 == 1:
            self.my_history.append(1)
            return 1
        elif len(self.signals)%3 == 2:
            self.my_history.append(0)
            return 0
        else:
            self.my_history.append(0)
            return 0
        
    def attack_action(self):
        self.my_history.append(1)
        return 1
    
    def research1(self):
        num_attack = 0.0
        for i in self.signals:
            if i == 1:
                num_attack += 1.0
        attack_percent = num_attack / (float(len(self.signals))+1)
        if attack_percent >= 0.5:
            return 1
        else:
            return 0
        
    def research2(self):
        for i in range(len(self.signals)-3):
            if self.signals[-i-1] == 1 and self.signals[-i-2] == 1 and self.signals[-i-3] == 1:
                return 1
        else:
            return 0
        
    def research3(self):
        num_attack = 20.0
        for i in self.signals:
            if i == 1:
                num_attack += 1.0
        attack_percent = num_attack / (float(len(self.signals))+1)
        if attack_percent >= 0.2:
            return 1
        else:
            return 0

    def play(self):
        research1_result = self.research1()
        research3_result = self.research3()
        if len(self.signals) <= 3:
            if research1_result == 0:
                return self.normal_action()
            else:
                return self.attack_action()
        else:
            prior_signal = self.signals[-1]
            research2_result = self.research2()
            if research2_result == 1:
                return self.attack_action()
            elif research1_result == 1:
                return self.attack_action()
            elif lem(self.myself) >= 20 and research3_result == 1:
                return self.attack_action()
            else:
                return self.normal_action()

    def get_signal(self, signal):
        self.signals.append(1)

In [14]:
# -*- coding: utf-8 -*-
from __future__ import division, print_function
import numpy as np
import unittest


In [15]:
#-*- encoding: utf-8 -*-
'''
サンプル戦略集
'''
import numpy as np


# テンプレート
class MyStrategy():
    def __init__(self, random_state=None):
        # RandomStateオブジェクトのインスタンスを受け取る
        # 確率変数を使いたい場合は、このインスタンスを使う
        if random_state is None:
            random_state = np.random.RandomState()
        self.random_state = random_state
        
        # 自分の行動の履歴
        self.my_history = []
        
        # 過去の全てのシグナル
        self.signals = []


    # 各ステージゲームの実行時に呼び出されるメソッド
    # その期の行動を0（=協調）, または1（=攻撃）のいずれかから選ぶ
    def play(self):
        # 第1期は協調
        if len(self.signals) < 1:
            self.my_history.append(0)
            return 0

        # 前期のシグナル
        prior_signal = self.signals[-1]

        # 前期のシグナルがBadの時、20%の割合でこちらも攻撃する
        epsilon = self.random_state.uniform()
        if epsilon > 0.8 and prior_signal == 1:
            self.my_history.append(1)
            return 1
        
        else:
            self.my_history.append(0)
            return 0


    # 各期のゲーム終了時に呼び出されるメソッド
    def get_signal(self, signal):
        # 前期のゲームのシグナルを受け取る
        # 受け取ったシグナルをシグナルの履歴に追加
        self.signals.append(signal)


# 常に協調
class AllC(object):
    def __init__(self, random_state=None):
        pass

    def play(self):
        return 0

    def get_signal(self, signal):
        pass


# 常に攻撃
class AllD():
    def __init__(self, random_state=None):
        pass

    def play(self):
        return 1

    def get_signal(self, signal):
        pass



# 最初は協調を続け、相手が1度でも攻撃してきたら以後ずっと攻撃
# imperfect monitoringの場合は、シグナルが1になれば、以降ずっと攻撃
class GrimTrigger():
    def __init__(self, random_state=None):
        # 相手が協力的かどうかのflag
        # 相手が1度でも攻撃してきたらFalseにする
        self.cooperation_flag = True

    def play(self):
        if self.cooperation_flag:
            return 0

        else:
            return 1

    def get_signal(self, signal):
        if signal == 1:
            self.cooperation_flag = False


# 協調と攻撃を交互に繰り返す
class Alternate():
    def __init__(self, random_state=None):
        # 次にどの手をだすかのflag
        # Trueなら次は0, Falseなら次は1を出す
        self.flag = True

    def play(self):
        if self.flag:
            return 0

        else:
            return 1

    def get_signal(self, signal):
        self.flag = not self.flag


# ランダム
class RandomStrategy():
    def __init__(self, random_state=None):
        if random_state is None:
            random_state = np.random.RandomState()
        self.random_state = random_state

    def play(self):
        return self.random_state.randint(0, 2)

    def get_signal(self, signal):
        pass


In [16]:
class TestStrategies(unittest.TestCase):
    def setUp(self):
        self.Strategies = [KatoStrategy] # ここに自作のclassを入れる
        self.case1 = "Signal is empty(period 1)"
        self.case2 = [0, 1]
        self.case3 = [1, 0]
        self.case4 = [0, 1, 0, 1, 0, 0, 1]

        self.seed = 11451
        self.RandomState = np.random.RandomState(self.seed)


    # case1を引数に渡してテスト
    def test1(self):
        print("testcase:", self.case1)
        for Strategy in self.Strategies:
            rst = Strategy(self.RandomState).play()
            self.assertIn(rst, (0, 1))


    # case2を引数に渡してテスト
    def test2(self):
        print("testcase:", self.case2)
        for Strategy in self.Strategies:
            S = Strategy(self.RandomState)
            S.get_signal(self.case2)
            rst = S.play()
            self.assertIn(rst, (0, 1))


    # case3を引数に渡してテスト
    def test3(self):
        print("testcase:", self.case3)
        for Strategy in self.Strategies:
            S = Strategy(self.RandomState)
            for signal in self.case3:
                S.get_signal(signal)
            
            rst = S.play()
            self.assertIn(rst, (0, 1))


    # case4を引数に渡してテスト
    def test4(self):
        print("testcase:", self.case4)
        for Strategy in self.Strategies:
            S = Strategy(self.RandomState)
            for signal in self.case4:
                S.get_signal(signal)
                rst = S.play()
            
            self.assertIn(rst, (0, 1))

In [17]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestStrategies)
unittest.TextTestRunner().run(suite)

E...

testcase: Signal is empty(period 1)
testcase: [0, 1]
testcase: [1, 0]
testcase: [0, 1, 0, 1, 0, 0, 1]



ERROR: test1 (__main__.TestStrategies)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-16-cef493bbcc84>", line 17, in test1
    rst = Strategy(self.RandomState).play()
  File "<ipython-input-13-72946d7bb436>", line 56, in play
    research1_result = self.research1()
  File "<ipython-input-13-72946d7bb436>", line 31, in research1
    attack_percent = num_attack / (float(len(self.signals)))
ZeroDivisionError: float division by zero

----------------------------------------------------------------------
Ran 4 tests in 0.006s

FAILED (errors=1)


<unittest.runner.TextTestResult run=4 errors=1 failures=0>